<a href="https://colab.research.google.com/github/moosenasser/ipynb-samples/blob/main/freeCodeCamp_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
VOCAB_SIZE = 1000

COL_HEADERS = ['type', 'message']

In [ ]:
# Store tsv's in dataframes.
raw_train_df = pd.read_csv(train_file_path, sep='\t', names=COL_HEADERS)
raw_validation_df = pd.read_csv(test_file_path, sep='\t', names=COL_HEADERS)

In [ ]:
# Convert categorical data in "type" column to numeric data.
raw_train_df['type'] = pd.factorize(raw_train_df['type'])[0]
raw_validation_df['type'] = pd.factorize(raw_validation_df['type'])[0]

In [ ]:
# Convert each pandas Dataframe to TensorFlow Dataset.

train_ds = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(raw_train_df['message'].values, tf.string),
            tf.cast(raw_train_df['type'].values, tf.int32)
        )
    )
)

validation_ds = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(raw_validation_df['message'].values, tf.string),
            tf.cast(raw_validation_df['type'].values, tf.int32)
        )
    )
)

In [ ]:
# Shuffle data and create batches.
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
validation_ds = validation_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Create text encoder, and adapt it to training dataset.

encoder = layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)

encoder.adapt(train_ds.map(lambda text, label: text))

In [ ]:
# Create model.
model = keras.Sequential([
    encoder,
    layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle variable sequence lengths.
        mask_zero=True),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

# Compile model.
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

# Train model.
model.fit(train_ds, epochs=10, validation_data=validation_ds,
          validation_steps=30)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])


def predict_message(pred_text_arg):
    numeric_sentiment_val = model.predict(np.array([pred_text_arg]))[0][0]
    spam_condition = round(numeric_sentiment_val) >= 0.0
    sms_class = 'spam' if spam_condition else 'ham'
    temp_prediction = [numeric_sentiment_val, sms_class]
    return temp_prediction


pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.


def test_predictions():
    test_messages = ["how are you doing today",
                     "sale today! to stop texts call 98912460324",
                     "i dont want to go. can we try it a different day? available sat",
                     "our new mobile video service is live. just install on your phone to start watching.",
                     "you have won £1000 cash! call to claim your prize.",
                     "i'll bring it tomorrow. don't forget the milk.",
                     "wow, is your arm alright. that happened to me one time too"
                     ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        temp_prediction = predict_message(msg)
        if temp_prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")


test_predictions()
